# 01 Sensitivity analysis of model parameters

In [1]:
import pickle

To make the `pickle` readable, we must add translation to the `sys.path`:

In [2]:
import sys
sys.path.append(r'../')
sys.path.append(r'../translation/')

In [3]:
import translation

In [4]:
filepath = "../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20211108_1103_20_ribosomes_0120s.p"

In [5]:
with open(filepath, 'rb') as f:
    results = pickle.load(f)

In [6]:
results.keys()

dict_keys(['proteome', 'peptide_bonds', 'transcriptome', 'timerange', 'timecourses', 'description', 'time_stamp', 'n_ribosomes', 'n_tRNA', 'duration'])

In [7]:
results['proteome']

Counter({'YGL030W': 1,
         'YLR325C': 2,
         'YPR043W': 2,
         'YOR234C': 1,
         'YPL037C': 1,
         'YAL003W': 2,
         'YHR193C': 1,
         'YGL103W': 1,
         'YLR043C': 1,
         'YDR224C': 2,
         'YDR382W': 1,
         'YKL152C': 2,
         'YLR075W': 3,
         'YOL086C': 1,
         'YLR340W': 1,
         'YNL145W': 1,
         'YLR293C': 1,
         'YAL038W': 3,
         'YPL131W': 1,
         'YKL081W': 1,
         'YOR293W': 2,
         'YAL012W': 1,
         'YKL216W': 1,
         'YOR369C': 1,
         'YJR123W': 2,
         'YGL123W': 1,
         'YOL039W': 1,
         'YDR500C': 1,
         'YOL109W': 1,
         'YNL135C': 1,
         'YHR001W': 1,
         'YDL007W': 1,
         'YOR020C': 1,
         'YHR020W': 1,
         'YDL055C': 1,
         'YBR011C': 1,
         'YOR065W': 1,
         'YBL045C': 1,
         'YNL178W': 1,
         'YML008C': 1,
         'YNL069C': 1,
         'YBL058W': 1,
         'YNL307C': 1,
         'Y

In [8]:
results['peptide_bonds']

20772

In [9]:
len(results['transcriptome'])

57508

In [10]:
results['timerange']

array([0.0000e+00, 5.0000e-02, 1.0000e-01, ..., 1.1985e+02, 1.1990e+02,
       1.1995e+02])

In [11]:
results['timecourses'].keys()

dict_keys(['proteins', 'ribos._bound', 'ribos._free', 'tRNA_bound', 'tRNA_free', 'ATP', 'AMP', 'GTP', 'GDP', 'peptide_bonds', 'tRNA_free_01', 'tRNA_free_02', 'tRNA_free_03', 'tRNA_free_04', 'tRNA_free_05', 'tRNA_free_06', 'tRNA_free_07', 'tRNA_free_08', 'tRNA_free_09', 'tRNA_free_10', 'tRNA_free_11', 'tRNA_free_12', 'tRNA_free_13', 'tRNA_free_14', 'tRNA_free_15', 'tRNA_free_16', 'tRNA_free_17', 'tRNA_free_18', 'tRNA_free_19', 'tRNA_free_20', 'tRNA_free_21', 'tRNA_free_22', 'tRNA_free_23', 'tRNA_free_24', 'tRNA_free_25', 'tRNA_free_27', 'tRNA_free_28', 'tRNA_free_29', 'tRNA_free_30', 'tRNA_free_31', 'tRNA_free_32', 'tRNA_free_33', 'tRNA_free_34', 'tRNA_free_35', 'tRNA_free_36', 'tRNA_free_37', 'tRNA_free_38', 'tRNA_free_39', 'tRNA_free_40', 'tRNA_free_41', 'tRNA_free_42', 'tRNA_free_43', 'YGL030W', 'YLR325C', 'YPR043W', 'YOR234C', 'YPL037C', 'YAL003W', 'YHR193C', 'YGL103W', 'YLR043C', 'YDR224C', 'YDR382W', 'YKL152C', 'YLR075W', 'YOL086C', 'YLR340W', 'YNL145W', 'YLR293C', 'YAL038W', 'YPL

In [12]:
results['description']

'updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah'

In [13]:
results['time_stamp']

'20211108_1103'

In [14]:
results['n_ribosomes']

20

In [15]:
results['n_tRNA']

2984788

In [16]:
results['duration']

120.95

## A. Ribosome count

In [5]:
filepath = {150000: "../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20211106_0316_150000_ribosomes_3600s.p",
            200000: "../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20211105_1049_200000_ribosomes_3600s.p",            
            250000: "../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20211106_2335_250000_ribosomes_3600s.p"}

In [7]:
results = {}
for ribocount in filepath.keys():
    with open(filepath[ribocount], 'rb') as f:
        print(ribocount)
        results[ribocount] = pickle.load(f)

150000
200000
250000


In [8]:
results.keys()

dict_keys([150000, 200000, 250000])

In [9]:
results[150000].keys()

dict_keys(['proteome', 'peptide_bonds', 'transcriptome', 'timerange', 'timecourses', 'description', 'time_stamp', 'n_ribosomes', 'n_tRNA', 'duration'])

In [32]:
results[200000]["timecourses"].keys()

dict_keys(['proteins', 'ribos._bound', 'ribos._free', 'tRNA_bound', 'tRNA_free', 'ATP', 'AMP', 'GTP', 'GDP', 'peptide_bonds', 'tRNA_free_01', 'tRNA_free_02', 'tRNA_free_03', 'tRNA_free_04', 'tRNA_free_05', 'tRNA_free_06', 'tRNA_free_07', 'tRNA_free_08', 'tRNA_free_09', 'tRNA_free_10', 'tRNA_free_11', 'tRNA_free_12', 'tRNA_free_13', 'tRNA_free_14', 'tRNA_free_15', 'tRNA_free_16', 'tRNA_free_17', 'tRNA_free_18', 'tRNA_free_19', 'tRNA_free_20', 'tRNA_free_21', 'tRNA_free_22', 'tRNA_free_23', 'tRNA_free_24', 'tRNA_free_25', 'tRNA_free_27', 'tRNA_free_28', 'tRNA_free_29', 'tRNA_free_30', 'tRNA_free_31', 'tRNA_free_32', 'tRNA_free_33', 'tRNA_free_34', 'tRNA_free_35', 'tRNA_free_36', 'tRNA_free_37', 'tRNA_free_38', 'tRNA_free_39', 'tRNA_free_40', 'tRNA_free_41', 'tRNA_free_42', 'tRNA_free_43'])

In [25]:
protnames = {}
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein']

for ribocount in results.keys():
    print(ribocount)
    protnames[ribocount] = results[ribocount]['timecourses'].keys()
    for fragment in blacklist:
        # TODO: perhaps better with filter
        protnames[ribocount] = [name for name in protnames[ribocount] if fragment not in name]

150000
200000
250000


In [26]:
protnames[150000]

['peptide_bonds']

In [29]:
results[250000]['timecourses'].keys()

dict_keys(['proteins', 'ribos._bound', 'ribos._free', 'tRNA_bound', 'tRNA_free', 'ATP', 'AMP', 'GTP', 'GDP', 'peptide_bonds', 'tRNA_free_01', 'tRNA_free_02', 'tRNA_free_03', 'tRNA_free_04', 'tRNA_free_05', 'tRNA_free_06', 'tRNA_free_07', 'tRNA_free_08', 'tRNA_free_09', 'tRNA_free_10', 'tRNA_free_11', 'tRNA_free_12', 'tRNA_free_13', 'tRNA_free_14', 'tRNA_free_15', 'tRNA_free_16', 'tRNA_free_17', 'tRNA_free_18', 'tRNA_free_19', 'tRNA_free_20', 'tRNA_free_21', 'tRNA_free_22', 'tRNA_free_23', 'tRNA_free_24', 'tRNA_free_25', 'tRNA_free_27', 'tRNA_free_28', 'tRNA_free_29', 'tRNA_free_30', 'tRNA_free_31', 'tRNA_free_32', 'tRNA_free_33', 'tRNA_free_34', 'tRNA_free_35', 'tRNA_free_36', 'tRNA_free_37', 'tRNA_free_38', 'tRNA_free_39', 'tRNA_free_40', 'tRNA_free_41', 'tRNA_free_42', 'tRNA_free_43'])